Advent of Code, Day 8

First, let's build our 2D grid. We'll read the data, strip whitespace, ensure each string character is an int, and then create the grid as a list of lists. Knowing that we'll need to look at each point on the grid from both a horizontal and vertical perspective, we'll also go ahead and create a transposed grid so we can leverage the same functions for it.

In [9]:
with open('data.csv') as file:
    lines = file.readlines()

grid = [list(map(int, l.strip())) for l in lines]
grid_transposed = [list(row) for row in zip(*grid)]

Let's also create a "forest of Trees" to represent each tree that we're going to look at. We'll use a namedtuple for readability and to set the attributes of the Tree in an easily readable fashion.

In [10]:
from collections import namedtuple
Tree = namedtuple("Tree", "x y height")

forest = []
for idxr, row in enumerate(grid):
    for idxc, item in enumerate(row):
        forest.append(Tree(idxc,idxr, int(grid[idxr][idxc])))

Next, let's build our sightlines to each tree. We'll slice the grid such that we have a tangible list of each of the cardinal sight lines to each tree.

In [11]:
trees_in_sight_line = {}

for tree in forest:
    trees_in_sight_line[(tree.x,tree.y)] = []
    trees_in_sight_line[(tree.x,tree.y)].append(grid[tree.y][:tree.x]) #left
    trees_in_sight_line[(tree.x,tree.y)].append(grid[tree.y][tree.x+1:]) #right
    trees_in_sight_line[(tree.x,tree.y)].append(grid_transposed[tree.x][:tree.y]) #up
    trees_in_sight_line[(tree.x,tree.y)].append(grid_transposed[tree.x][tree.y+1:]) #down

Finally, let's evaluate the visbility of the tree by checking the sightlines to determine if the tree we're interested in is visible from outside the forest.

In [12]:
visibility = {}

def is_tree_on_edge(tree_to_check):
    for sight_line in trees_in_sight_line[(tree_to_check.x, tree_to_check.y)]:
        if not sight_line:
            return True

def is_tree_tallest_along_a_sightline(tree_to_check):
    for sight_line in trees_in_sight_line[(tree_to_check.x, tree_to_check.y)]:
        if tree_to_check.height > max(sight_line):
            return True

def is_tree_visible(tree_to_check):
    visibility[tree_to_check] = is_tree_on_edge(tree_to_check) or is_tree_tallest_along_a_sightline(tree_to_check) or False

for tree in forest:
    is_tree_visible(tree)

sum(visibility.values())

1798

Part Two

In part two, our vantage point for the trees changes to looking from the inside out. As a result, we'll need to reverse the order of a couple of our list in order to change our definition of "sight lines" to match the new point of view.

In [13]:
trees_in_sight_line = {}

for tree in forest:
    trees_in_sight_line[(tree.x,tree.y)] = []
    left = grid[tree.y][:tree.x]
    left.reverse()
    trees_in_sight_line[(tree.x,tree.y)].append(left) #left
    trees_in_sight_line[(tree.x,tree.y)].append(grid[tree.y][tree.x+1:]) #right
    # looking up
    up = grid_transposed[tree.x][:tree.y]
    up.reverse()
    trees_in_sight_line[(tree.x,tree.y)].append(up) #up
    trees_in_sight_line[(tree.x,tree.y)].append(grid_transposed[tree.x][tree.y+1:]) #down

Next, we'll need to define a function that allows us to travese the list of trees in order to count the number that will end up in our viewshed, ending (and including) the first tree we find that is higher than ours.

In [14]:
def takewhile(height, iterable):
    for x in iterable:
        if height > x:
            yield 1
        elif height == x:
            yield 1
            break
        elif height < x:
            yield 1
            break

Finally, we'll compute our view score based on the method from the problem.

In [15]:
import math

tree_house_scores = {}

for tree in forest:
    sightline_score = []
    for sight_line in trees_in_sight_line[(tree.x, tree.y)]:
        sightline_score.append(sum(takewhile(tree.height, sight_line)))

    tree_house_scores[(tree.x, tree.y)] = math.prod(sightline_score)

max(tree_house_scores.values())

259308